In [1]:
import matplotlib.pyplot as plt
import scipy.constants as sc
import numpy as np
import xtrack as xt
import xcoll as xc
import xobjects as xo
import xpart as xp
from pathlib import Path
import lossmaps as lm
import cernlayoutdb as layout
from cpymad.madx import Madx

 If you are on SWAN and did not expect this warning, then restart and load the 'NXCals Pro' software stack.


# Generating line from madx sequence WITH ERRORS - so non linearities

In [5]:
path_lines = Path('../injection_lines/')
line = xt.Line.from_json(Path(path_lines, 'injection_thin_approx_ap_with_deferred_expressions.json'))#'injection_lines/injection_thin_approx_ap_with_deferred_expressions.json')

Loading line from dict:   0%|          | 0/31362 [00:00<?, ?it/s]

Done loading line from dict.           


In [9]:
OPTICSDIR = "/eos/user/l/lpauwels/acc-models-sps/"
OPTICSDIR = "cernbox/acc-models-sps"

p = 26  # beam momentum (GeV/c)

mad = Madx(stdout=True)

#mad.call(file=f"{OPTICSDIR}/SPS_LS2_2020-05-26.seq")
mad.call(file="acc_models/EYETS 2024-2025.seq")
mad.call(file=f"{OPTICSDIR}/strengths/lhc_q20.str") # change this depending on which optics you want
mad.call(file=f"{OPTICSDIR}/toolkit/macro.madx")

mad.command.beam(particle="PROTON", pc=p, charge=1)
mad.input("BRHO = BEAM->PC * 3.3356;")

mad.use(sequence='sps')
slice_kwargs = {'rbend':4, 'quadrupole':4, 'sbend':4} # here I am slicing every bend and quad into 4, change if you don't want so many slices
for class_, slice_ in slice_kwargs.items():
    mad.select(flag='makethin', class_=class_, slice_=slice_)
mad.makethin(sequence='sps', style='teapot', makedipedge=True)

mad.use(sequence="sps")
mad.input('exec, sps_set_tune_weights_q20;')
mad.input('exec, sps_set_chroma_weights_q20;')
mad.input('exec, sps_define_quad_knobs;')
mad.input('exec, sps_define_sext_knobs;')

mad.call(file=f"{OPTICSDIR}/sps_setMultipoles_upto7.cmd")
mad.input('exec, set_Multipoles_26GeV;')
mad.call(file=f"{OPTICSDIR}/sps_assignMultipoles_upto7.cmd")
mad.input('exec, AssignMultipoles;')

line = xt.Line.from_madx_sequence(mad.sequence["sps"], deferred_expressions=True, 
                                    apply_madx_errors=True, 
                                    install_apertures=False, 
                                    allow_thick=False)

line.particle_ref = xt.Particles(mass0=xp.PROTON_MASS_EV,
                                    gamma0=mad.sequence["sps"].beam.gamma)

line.vv['v200'] = 1e6 # total RF voltage
line.vv['freq200'] = 200e6 # RF frequency
line.vv['lag200'] = 180 # RF phase
line.element_refs['actcse.31632'].voltage = line.vars['v200']
line.element_refs['actcse.31632'].frequency = line.vars['freq200']
line.element_refs['actcse.31632'].lag = line.vars['lag200']


line.to_json("sps_lhc_q20_rf.json") # change the name of the line to whatever you want


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Darwin)      +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2025.04.07 11:44:31      +
  ++++++++++++++++++++++++++++++++++++++++++++




  /**********************************************************************************

  *

  * SPS Ring version (draft)EYETS 2024-2025 in MAD X SEQUENCE format

  * Generated the 18-MAR-2025 22:25:54 from Layout

  *

  ***********************************************************************************/







/************************************************************************************/

/*                       STRENGTH CONSTANTS                                         */

/************************************************************************************/



kMBA                := 0.008445141542;

kMBB                := 0.008445141542;



/***************************************************

+=+=+= fatal: cannot open input file: cernbox/acc-models-sps/strengths/lhc_q20.str


RuntimeError: MAD-X has stopped working!

# Generating thick line with deferred expressions

In [ ]:
#OPTICSDIR = "/eos/user/l/lpauwels/acc-models-sps/"
OPTICSDIR = "line_generation/acc_models"

p = 26  # beam momentum (GeV/c)

mad = Madx(stdout=True)

# mad.call(file=f"{OPTICSDIR}/SPS_LS2_2020-05-26.seq")
mad.call(file=f"{OPTICSDIR}/SPS_LS2_2025-03-05.seq")
mad.call(file=f"{OPTICSDIR}/strengths/lhc_q20.str") # change this depending on which optics you want
mad.call(file=f"{OPTICSDIR}/toolkit/macro.madx")

mad.command.beam(particle="PROTON", pc=p, charge=1)
mad.input("BRHO = BEAM->PC * 3.3356;")

mad.use(sequence="sps")
mad.input('exec, sps_set_tune_weights_q20;')
mad.input('exec, sps_set_chroma_weights_q20;')
mad.input('exec, sps_define_quad_knobs;')
mad.input('exec, sps_define_sext_knobs;')

line = xt.Line.from_madx_sequence(mad.sequence["sps"], deferred_expressions=True, 
                                    apply_madx_errors=False, 
                                    install_apertures=False, 
                                    allow_thick=True)

line.particle_ref = xt.Particles(mass0=xp.PROTON_MASS_EV,
                                    gamma0=mad.sequence["sps"].beam.gamma)

line.vv['v200'] = 1e6 # total RF voltage
line.vv['freq200'] = 200e6 # RF frequency
line.vv['lag200'] = 180 # RF phase
line.element_refs['actcse.31632'].voltage = line.vars['v200']
line.element_refs['actcse.31632'].frequency = line.vars['freq200']
line.element_refs['actcse.31632'].lag = line.vars['lag200']


line.to_json("injection_lines/injection_thick_no_ap_with_deffered_expressions.json") 


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2025.02.24 11:48:35      +
  ++++++++++++++++++++++++++++++++++++++++++++




  /**********************************************************************************

  *

  * SPS Ring version (draft)LS2 in MAD X SEQUENCE format

  * Generated the 26-MAY-2020 14:48:33 from Layout

  *

  ***********************************************************************************/









/************************************************************************************/

/*                       TYPES DEFINITION                                           */

/************************************************************************************/



//---------------------- COLLIMATOR     ---------------------------------------------

BSHV      : COLLIMATOR  , L := .45;       ! beam scraper,

Converting sequence "sps":   0%|          | 0/3794 [00:00<?, ?it/s]


  Number of warnings: 0

  ++++++++++++++++++++++++++++++++++++++++++++
  +          MAD-X finished normally         +
  ++++++++++++++++++++++++++++++++++++++++++++
